# Project 2 

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Main 

In [6]:
'''
DATA_Ux_PATH = 'data/u1.csv'
DATA_Uy_PATH = 'data/u2.csv'
Ux, ids_Ux= load_csv_data(DATA_Ux_PATH, sub_sample=False)
Uy, ids_Uy = load_csv_data(DATA_Uy_PATH, sub_sample=False)
'''

'\nDATA_TRAIN_PATH = \nDATA_TEST_PATH = \ny, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=False)\ny_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH, sub_sample=False)\nfeatures_name = np.genfromtxt(\'../data/train.csv\', delimiter=",", dtype=str,max_rows=1)[2:]\n'

In [ ]:
print(Ux.shape)

In [ ]:
ratio_pts = 0.1
ratio_time = 0.5

new_Ux, new_Uy = sample(Ux, Uy, ratio_pts, ratio_time)

In [ ]:
print(new_Ux.shape)

1. modeliser mathématiquement l’écoulement sanguin à l’aide de PDE dépendante de 2-3 paramètres physique
2. simuler l’écoulement par un code d’éléments fini ou similaire
3. générer beaucoup de solutions avec une grande nombre de paramètres différents. (Les 2-3, pris de façon aléatoire)
4. a. utiliser les solutions numérique pour établir un auto encoder qui au milieu n’ai que 2-5 hyper-paramètres libres  
b. quel erreur on obtient ? Est-ce possible de réduire le nombre d’hyper-paramètres ?
5. étudier s’il y a une rélation entre les 2-5 hyper-paramètre et les paramètres physique  
a. à l’aide de statistiques
5. b. à l’aide d’un DNN (différent de 4a)
6. (optionnel) faire un DNN entre l’input de 4a et output les paramêtres physique. Et/ou l’inverse.
7. Discussion et conclusions 

## 1. Data exploration

## 2. Feature processing

## 3. Solution generation

## 4. Auto-encoder

## 5. Hyper-parameters and physics parameters relationship

## 6. Discussion & conclusion